In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="uhtPWClvIGgUa3WaB0k6")
project = rf.workspace("ml-lmvau").project("birddetector-wrvb1-v20r0")
version = project.version(1)
dataset = version.download("yolov11")

In [ ]:
dataset.location

In [ ]:
!yolo task=detect mode=train data={dataset.location}/data.yaml model="yolo11l.pt" epochs=100 imgsz=640 batch=16 plots=True save=True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import pandas as pd
from PIL import Image
from tqdm import tqdm
from ultralytics import YOLO

# ===========================
# CONFIGURATION
# ===========================
MODEL_PATH = "/content/drive/MyDrive/bdma-07-competition/best.pt"  # Path to trained YOLO11L model
ROOT_DIR = "/content/drive/MyDrive/bdma-07-competition/BDMA7_project_files/"
SAVE_DIR = "/content/drive/MyDrive/bdma-07-competition/BDMA7_project_files/cropped_images/"
RESIZE_SHAPE = (448, 448)  # Resize cropped images to 448x448

# Paths to metadata files
train_csv = os.path.join(ROOT_DIR, "train_metadata.csv")
val_csv = os.path.join(ROOT_DIR, "val_metadata.csv")
train_img_dir = os.path.join(ROOT_DIR, "train_images")
val_img_dir = os.path.join(ROOT_DIR, "val_images")
test_img_dir = os.path.join(ROOT_DIR, "test_images/mistery_cat")

# ===========================
# LOAD YOLO MODEL
# ===========================
model = YOLO(MODEL_PATH)

# ===========================
# HELPER FUNCTION: CROP, RESIZE, AND SAVE IMAGES
# ===========================
def crop_resize_save_images(image_paths, save_subdir, is_test=False):
    for img_path in tqdm(image_paths, desc=f"Processing {save_subdir} images"):
        try:
            filename = img_path
            if not is_test:
                class_name = filename.split('/')[7]  # Extract class from folder structure
                class_save_dir = os.path.join(SAVE_DIR, save_subdir, class_name)
                os.makedirs(class_save_dir, exist_ok=True)
                save_path = os.path.join(class_save_dir, filename.split('/')[8])  # Keep original name
            else:
                test_save_dir = os.path.join(SAVE_DIR, save_subdir)
                os.makedirs(test_save_dir, exist_ok=True)
                save_path = os.path.join(test_save_dir, filename.split('/')[8])  # Keep original name

            # Read image
            image = cv2.imread(img_path)
            if image is None:
                print(f"Warning: Could not read {img_path}")
                continue

            # Run YOLO inference
            results = model(img_path)

            for result in results:
                boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes [x1, y1, x2, y2]
                if len(boxes) == 0:
                    print(f"Warning: No objects detected in {img_path}, skipping cropping.")
                    continue  # Skip if no objects detected

                # Use the first detected box (assuming one object per image)
                x1, y1, x2, y2 = map(int, boxes[0])

                # Crop the detected region
                cropped_img = image[y1:y2, x1:x2]

                # Convert to PIL format, resize, and save
                cropped_img_pil = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
                resized_img_pil = cropped_img_pil.resize(RESIZE_SHAPE, Image.Resampling.LANCZOS)
                resized_img_pil.save(save_path)

        except Exception as e:
            print(f"Error processing {img_path}: {e}")

# ===========================
# PROCESS TRAIN, VAL, TEST IMAGES
# ===========================

# Load train and val metadata
train_metadata = pd.read_csv(train_csv)
val_metadata = pd.read_csv(val_csv)

# Get full image paths
train_images = [os.path.join(train_img_dir, img) for img in train_metadata.iloc[:, 0].tolist()]
val_images = [os.path.join(val_img_dir, img) for img in val_metadata.iloc[:, 0].tolist()]
test_images = [os.path.join(test_img_dir, img) for img in os.listdir(test_img_dir) if img.endswith((".jpg", ".png"))]

print(train_images[0].split("/")[7])

# Process images
crop_resize_save_images(train_images, "train_images")
crop_resize_save_images(val_images, "val_images")
crop_resize_save_images(test_images, "test_images/mistery_cat", is_test=True)

print("✅ Cropped and resized images saved successfully!")

In [ ]:
# done